Файлы в формате kml и kmz позволяют загрузить в Google Earth Pro список объектов с координатами и отобразить их на карте. Что может быть полезно, например, для определения в каком населенном пункте находится станция, т.к. в наших метаданных далеко не во всех станциях указан населенный пункт в которых они раполагаются. Также, в рамках нашего проекта такие файлы можно, например, использовать для отображения индексов качества воздуха для каждой станции, если предварительно создать и использовать маркеры разных цветов, соответствующих цветам шкалы индекса качества воздуха.<br><br>
В качестве основы был взят файл AirBase_v5.kmz. Из него распаковкой был получен kml-файл, который был нарезан на элементы-шаблоны, далее в этих  шаблонах вручную были размечены необходимые переменные. Для удобства все основные элементы были сохранены в файл "kml_elements.csv". В случае если необходимо создать более сложную древовидную структуру, то в элемент folder можно вставлять подпапки с отступом в два пробела, которые, аналогичным образом, можно заполнять другими подпапками до получения необходимой структуры.<br><br>
В данной работе kml_exporter был использован для создания kml-файла с информацией о всех станциях входящих в новейшие метаданные.

In [1]:
import datetime
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
### Import previously created kml templates

# with open('mark2.txt', 'r', encoding='utf-8') as f:
#     temp_page = f.readlines()
# with open('folder2begin.txt', 'r', encoding='utf-8') as f:
#     folder_start = f.readlines()
# with open('folder2end.txt', 'r', encoding='utf-8') as f:
#     folder_end = f.readlines()
# with open('folder3begin.txt', 'r', encoding='utf-8') as f:
#     subfolder_start = f.readlines()
# with open('folder3end.txt', 'r', encoding='utf-8') as f:
#     subfolder_end = f.readlines()
# with open('doc2begin.txt', 'r', encoding='utf-8') as f:
#     doc_begin = f.readlines()
# with open('doc2end.txt', 'r', encoding='utf-8') as f:
#     doc_end = f.readlines()
# with open('AirBase_v5.kml', 'r', encoding='utf-8') as f:
#     AirBase = f.readlines()

In [3]:
### Create and export kml_df with containing kml templates

# kml1 = {'placemark': (''.join(temp_page) + '\n'), \
#         'folder_top': ''.join(folder_start), \
#         'folder_bot': ('\n' + ''.join(folder_end) + '\n'), \
#         'subfolder_top': ('\n' + ''.join(subfolder_start) + '\n'), \
#         'subfolder_bot': ''.join(subfolder_end), \
#         'doc_top': (''.join(doc_begin) + '\n'), \
#         'doc_bot': ''.join(doc_end), \
#         'airbase_v5_kml': ''.join(AirBase)}
# kml_df = pd.DataFrame(0, index=range(8), columns=range(0))
# kml_df['ele'] = kml1.keys()
# kml_df['val'] = kml1.values()
# kml_df.to_csv("kml_elements.csv", index=False, encoding='utf-8-sig')

In [4]:
### KML exporter function
### Exports stations grouped by countries
### Each country has subfolders corresponding to 3 categories: inactive, active, active with unspecified location
### Can be modified and tweaked to group by something else, if no extra subfolder levels are required
### If you need extra subfolder levels you need to edit and add new kml templates into kml_df

def kml_exporter(df, Username='Adele Kim', Githublink='https://github.com/Adele-Kim') -> None:
    kml_df = pd.read_csv('kml_elements.csv')
    folder_list = []
    stat_count = 0
    df1 = df.drop_duplicates(['AirQualityStation'], keep='first')
    for element in list(df1['Countrycode'].unique()):
        dff = df1[df1['Countrycode'] ==element].reset_index(drop=True)
        temp_page_inactive_list = []
        temp_page_active_list = []
        temp_page_new_list = []
        for i in range(len(list(dff['AirQualityStation']))):
            DateEnd = ''
            a_status = ''
            AirQualityStation = dff.loc[i, 'AirQualityStation'] 
            LocalCode = dff.loc[i, 'LocalCode']
            StationName = dff.loc[i, 'StationName']
            try:
                ObservationDateBegin = dff.loc[i, 'ObservationDateBegin'][0:10]
            except:
                ObservationDateBegin = ''
            status1 = [dff.loc[i, 'ObservationDateEnd']!=dff.loc[i, 'ObservationDateEnd']][0]
            status2 = [dff.loc[i, 'LocalCode']!=dff.loc[i, 'LocalCode']][0]
            if (status1==True) or (status2==True):
                try:
                    DateEnd = dff.loc[i, 'ObservationDateEnd'][0:10]
                except:
                    DateEnd = ''
                a_status = 'inactive station'
            elif dff.loc[i, 'StationCity']!=dff.loc[i, 'StationCity']:
                a_status = 'active station without location'
                DateEnd = ''
            else:
                a_status = 'active station'
                DateEnd = ''
            ObservationDateEnd = DateEnd
            active_status = a_status
            AirQualityStationType = dff.loc[i, 'AirQualityStationType']
            AirQualityStationArea = dff.loc[i, 'AirQualityStationArea']
            Longitude = format(dff.loc[i, 'Longitude'], '.6f')
            Latitude = format(dff.loc[i, 'Latitude'], '.6f')
            Altitude = format(dff.loc[i, 'Altitude'])
            my_str = (kml_df['val'][0]).format(AirQualityStation=AirQualityStation, \
                                               LocalCode=LocalCode, StationName=StationName, \
                                               ObservationDateBegin=ObservationDateBegin, \
                                               ObservationDateEnd=ObservationDateEnd, \
                                               active_status=active_status, \
                                               AirQualityStationType=AirQualityStationType, \
                                               AirQualityStationArea=AirQualityStationArea, \
                                               Longitude=Longitude, Latitude=Latitude, Altitude=Altitude)
            if active_status== 'inactive station':
                temp_page_inactive_list.append(my_str)
            elif active_status== 'active station':
                temp_page_active_list.append(my_str)
            elif active_status== 'active station without location':
                temp_page_new_list.append(my_str)
        subfolder_text1 = kml_df['val'][3] + ''.join(temp_page_inactive_list) + kml_df['val'][4]
        subfolder_text1 = subfolder_text1.format(aa_status='inactive', StatusNumber=len(temp_page_inactive_list))
        subfolder_text2 = kml_df['val'][3] + ''.join(temp_page_active_list) + kml_df['val'][4]
        subfolder_text2 = subfolder_text2.format(aa_status='active', StatusNumber=len(temp_page_active_list))
        subfolder_text3 = kml_df['val'][3] + ''.join(temp_page_new_list) + kml_df['val'][4]
        subfolder_text3 = subfolder_text3.format(aa_status='active_unspecified_location', StatusNumber=len(temp_page_new_list))
        subfolder_text_final = subfolder_text1 + subfolder_text2 + subfolder_text3
        StationsNumber = len(temp_page_inactive_list)+len(temp_page_active_list)+len(temp_page_new_list)
        stat_count = stat_count + StationsNumber
        folder_text = kml_df['val'][1] + subfolder_text_final + kml_df['val'][2]
        folder_text = folder_text.format(Countrycode=element, StationsNumber=StationsNumber)
        folder_list.append(folder_text)
    document_text = kml_df['val'][5] + ''.join(folder_list) + kml_df['val'][6]
    TotalStationsNum = stat_count
    Date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    UserName, GithubLink = Username, Githublink
    document_text = document_text.format(TotalStationsNum=TotalStationsNum, Date=Date, \
                                         UserName=UserName, GithubLink=GithubLink)
    filename = datetime.datetime.now().strftime('%Y_%m_%d_%H') + '_' + 'stations' + '.kml'
    with open(filename, 'w', encoding='utf-8') as the_file:
        the_file.write(document_text)

In [5]:
new_metadata = pd.read_csv('final_metadata.csv', low_memory=False)

In [6]:
kml_exporter(new_metadata)